# qBraid-sdk Transpiler

In [1]:
import braket
from braket.circuits.gate import Gate as BraketGate
from braket.circuits.instruction import Instruction as BraketInstruction
import cirq
import qiskit
from qiskit.circuit import Parameter
import numpy as np
from qbraid.transpiler.transpiler import qbraid_wrapper

### Transpile circuits between braket, cirq, and qiskit

1. Create circuit using any qBraid supported package, e.g. braket

In [2]:
braket_circuit_0 = braket.circuits.Circuit()

instructions = [
    BraketInstruction(BraketGate.H(), 0),
    BraketInstruction(BraketGate.X(), 1),
    BraketInstruction(BraketGate.Y(), 2),
    BraketInstruction(BraketGate.Z(), 1),
    BraketInstruction(BraketGate.S(), 0),
    BraketInstruction(BraketGate.Si(), 1),
    BraketInstruction(BraketGate.T(), 2),
    BraketInstruction(BraketGate.Ti(), 1),
    BraketInstruction(BraketGate.I(), 0),
    BraketInstruction(BraketGate.V(), 0),
    BraketInstruction(BraketGate.Vi(), 2),
    BraketInstruction(BraketGate.PhaseShift(np.pi), 2),
    BraketInstruction(BraketGate.Rx(np.pi), 0),
    BraketInstruction(BraketGate.Ry(np.pi), 1),
    BraketInstruction(BraketGate.Rz(np.pi / 2), 2),
    BraketInstruction(BraketGate.CNot(), [1, 0]),
    BraketInstruction(BraketGate.Swap(), [1, 2]),
    BraketInstruction(BraketGate.ISwap(), [1, 2]),
    BraketInstruction(BraketGate.PSwap(np.pi), [0, 1]),
    BraketInstruction(BraketGate.CY(), [0, 1]),
    BraketInstruction(BraketGate.CZ(), [1, 0]),
    BraketInstruction(BraketGate.CPhaseShift(np.pi / 4), [2, 0]),
    BraketInstruction(BraketGate.XX(np.pi), [0, 1]),
    BraketInstruction(BraketGate.XY(np.pi), [0, 1]),
    BraketInstruction(BraketGate.YY(np.pi), [0, 1]),
    BraketInstruction(BraketGate.ZZ(np.pi), [0, 1]),
    BraketInstruction(BraketGate.CCNot(), [0, 1, 2]),
]

for inst in instructions:
    braket_circuit_0.add_instruction(inst)

In [3]:
print("braket circuit\n")
print(braket_circuit_0)

braket circuit

T  : |0|1|2 |     3     |   4    |5| 6  |  7  |     8     |9|10|     11     |   12   |   13   |   14   |   15   |16|
                                                                                                                    
q0 : -H-S-I--V-----------Rx(3.14)-X------------PSWAP(3.14)-C-Z--PHASE(0.785)-XX(3.14)-XY(3.14)-YY(3.14)-ZZ(3.14)-C--
                                  |            |           | |  |            |        |        |        |        |  
q1 : -X-Z-Si-Ti----------Ry(3.14)-C-SWAP-ISWAP-PSWAP(3.14)-Y-C--|------------XX(3.14)-XY(3.14)-YY(3.14)-ZZ(3.14)-C--
                                    |    |                      |                                                |  
q2 : -Y-T-Vi-PHASE(3.14)-Rz(1.57)---SWAP-ISWAP------------------C------------------------------------------------X--

T  : |0|1|2 |     3     |   4    |5| 6  |  7  |     8     |9|10|     11     |   12   |   13   |   14   |   15   |16|


2. Apply the qBraid wrapper

In [4]:
qbraid_circuit_0 = qbraid_wrapper(braket_circuit_0)

3. Transpile circuit to any other qBraid supported package, i.e. cirq, qiskit

In [5]:
cirq_circuit_0 = qbraid_circuit_0.transpile(package="cirq")

print("cirq circuit\n")
print(cirq_circuit_0)

cirq circuit

0: ───H───S───U───U───Rx(π)──────X───────────────U───@───@───@────────U───U───U───U───@───
                                 │               │   │   │   │        │   │   │   │   │
1: ───X───Z───U───U───Ry(π)──────@───×───iSwap───U───Y───@───┼────────U───U───U───U───@───
                                     │   │                   │                        │
2: ───Y───T───U───Z───Rz(0.5π)───────×───iSwap───────────────@^0.25───────────────────X───


In [6]:
qiskit_circuit_0 = qbraid_circuit_0.transpile("qiskit")

print("qiskit circuit\n")
print(qiskit_circuit_0)

qiskit circuit

      ┌───┐┌───┐ ┌───┐   ┌────┐  ┌───────┐ ┌───┐             ┌──────────┐     »
q0_0: ┤ H ├┤ S ├─┤ I ├───┤ √X ├──┤ RX(π) ├─┤ X ├─────────────┤0         ├──■──»
      ├───┤├───┤┌┴───┴┐ ┌┴────┤  ├───────┤ └─┬─┘   ┌────────┐│  Unitary │┌─┴─┐»
q0_1: ┤ X ├┤ Z ├┤ SDG ├─┤ TDG ├──┤ RY(π) ├───■───X─┤0       ├┤1         ├┤ Y ├»
      ├───┤├───┤├─────┴┐├─────┴┐┌┴───────┴┐      │ │  ISWAP │└──────────┘└───┘»
q0_2: ┤ Y ├┤ T ├┤ √XDG ├┤ P(π) ├┤ RZ(π/2) ├──────X─┤1       ├─────────────────»
      └───┘└───┘└──────┘└──────┘└─────────┘        └────────┘                 »
«                  ┌─────────┐┌──────────┐┌─────────┐             
«q0_0: ─■──■───────┤0        ├┤0         ├┤0        ├─■────────■──
«       │  │       │  RXX(π) ││  Unitary ││  RYY(π) │ │ZZ(π)   │  
«q0_1: ─■──┼───────┤1        ├┤1         ├┤1        ├─■────────■──
«          │P(π/4) └─────────┘└──────────┘└─────────┘        ┌─┴─┐
«q0_2: ────■─────────────────────────────────────────────────┤ X ├
«                     

### Test transpile circuit equivalence

In [7]:
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)

op_h = cirq.H(q0) ** 1.8
op_cnot = cirq.CNOT(q0, q1)
op_z = cirq.Z(q1)
op_t = cirq.T(q0)
op_s = cirq.S(q1)

h = cirq.H.controlled(1)
op_controlled_h = h(q0, q1)

cirq_circuit_1 = cirq.Circuit()
cirq_circuit_1.append(op_h)
cirq_circuit_1.append(cirq.H(q0))
cirq_circuit_1.append(cirq.X(q1))
cirq_circuit_1.append(op_cnot)
cirq_circuit_1.append(op_z)
cirq_circuit_1.append(op_s)
cirq_circuit_1.append(op_t)
cirq_circuit_1.append(op_controlled_h)

In [8]:
print("cirq circuit 1\n")
print(cirq_circuit_1)

cirq circuit 1

0: ───H^-0.2───H───@───T───────@───
                   │           │
1: ───X────────────X───Z───S───H───


Apply qBraid wrapper and transpile to qiskit

In [9]:
qbraid_circuit_1 = qbraid_wrapper(cirq_circuit_1)

In [10]:
qiskit_circuit_1 = qbraid_circuit_1.transpile("qiskit")

In [11]:
print("qiskit circuit 1\n")
print(qiskit_circuit_1)

qiskit circuit 1

      ┌─────────┐┌───┐     ┌───┐     ┌──────────┐
q1_0: ┤ Unitary ├┤ H ├──■──┤ T ├─────┤0         ├
      └──┬───┬──┘└───┘┌─┴─┐├───┤┌───┐│  Unitary │
q1_1: ───┤ X ├────────┤ X ├┤ Z ├┤ S ├┤1         ├
         └───┘        └───┘└───┘└───┘└──────────┘


Apply qBraid wrapper to new qiskit circuit and transpile back to cirq

In [12]:
cirq_circuit_2 = qbraid_wrapper(qiskit_circuit_1).transpile("cirq")

In [13]:
print("cirq circuit 2\n")
print(cirq_circuit_2)

cirq circuit 2

0: ───U───H───@───T───────U───
              │           │
1: ───X───────X───Z───S───U───


Check equivalence

In [14]:
np.allclose(cirq_circuit_1.unitary(), cirq_circuit_2.unitary())

True

### Transpile parameterized circuits

Create a qiskit parameterized circuit

In [15]:
theta = Parameter("\u03B8")

qiskit_param_circuit = qiskit.QuantumCircuit(5, 1)
qiskit_param_circuit.rz(np.pi / 4, range(5))
qiskit_param_circuit.h(0)

for i in range(3):
    qiskit_param_circuit.cx(i, i + 1)

qiskit_param_circuit.rz(theta, range(2))

for i in reversed(range(2)):
    qiskit_param_circuit.cx(i, i + 1)
    
qiskit_param_circuit.h(0)
qiskit_param_circuit.measure(0, 0)

In [16]:
print("qiskit parameterized circuit\n")
print(qiskit_param_circuit)

qiskit parameterized circuit

     ┌─────────┐┌───┐     ┌───────┐                   ┌───┐┌─┐
q_0: ┤ RZ(π/4) ├┤ H ├──■──┤ RZ(θ) ├────────────────■──┤ H ├┤M├
     ├─────────┤└───┘┌─┴─┐└───────┘┌───────┐     ┌─┴─┐└───┘└╥┘
q_1: ┤ RZ(π/4) ├─────┤ X ├────■────┤ RZ(θ) ├──■──┤ X ├──────╫─
     ├─────────┤     └───┘  ┌─┴─┐  └───────┘┌─┴─┐└───┘      ║ 
q_2: ┤ RZ(π/4) ├────────────┤ X ├──────■────┤ X ├───────────╫─
     ├─────────┤            └───┘    ┌─┴─┐  └───┘           ║ 
q_3: ┤ RZ(π/4) ├─────────────────────┤ X ├──────────────────╫─
     ├─────────┤                     └───┘                  ║ 
q_4: ┤ RZ(π/4) ├────────────────────────────────────────────╫─
     └─────────┘                                            ║ 
c: 1/═══════════════════════════════════════════════════════╩═
                                                            0 


Apply qBraid wrapper and transpile to cirq

In [17]:
qbraid_param_circuit = qbraid_wrapper(qiskit_param_circuit)
cirq_param_circuit = qbraid_param_circuit.transpile("cirq")
print("cirq parameterized circuit\n")
print(cirq_param_circuit)

cirq parameterized circuit

0: ───Rz(0.25π)───H───@───Rz(θ)───────────────@───H───M───
                      │                       │
1: ───Rz(0.25π)───────X───@───────Rz(θ)───@───X───────────
                          │               │
2: ───Rz(0.25π)───────────X───────@───────X───────────────
                                  │
3: ───Rz(0.25π)───────────────────X───────────────────────

4: ───Rz(0.25π)───────────────────────────────────────────


### Run circuits on any Device